In [24]:
import torch
import math


In [25]:

def haar_matrix(n, dtype=torch.float32, device='cpu'):
    """
    构造 n x n 的 Haar 变换矩阵，n 必须为 2 的幂次。
    """
    if n < 2 or (n & (n - 1)) != 0:
        raise ValueError("Haar matrix size must be a power of 2 and >= 2")

    def haar_recursive(n):
        if n == 2:
            return torch.tensor([[1., 1.],
                                 [1., -1.]])
        else:
            H_n_1 = haar_recursive(n // 2)
            top = torch.kron(H_n_1, torch.tensor([[1., 1.]]))
            bottom = torch.kron(torch.eye(len(H_n_1)), torch.tensor([[1., -1.]]))
            return torch.cat([top, bottom], dim=0)

    H = haar_recursive(n) / math.sqrt(n)
    return H.to(dtype=dtype, device=device)


In [26]:
a = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
print("输入信号：")
print(a)
print("haar 变换结果：")
H = haar_matrix(4, dtype=torch.float32)
print(H)
print("Haar 变换矩阵：")
print(H @ a)

输入信号：
tensor([1., 2., 3., 4.])
haar 变换结果：
tensor([[ 0.5000,  0.5000,  0.5000,  0.5000],
        [ 0.5000,  0.5000, -0.5000, -0.5000],
        [ 0.5000, -0.5000,  0.0000, -0.0000],
        [ 0.0000, -0.0000,  0.5000, -0.5000]])
Haar 变换矩阵：
tensor([ 5.0000, -2.0000, -0.5000, -0.5000])


In [27]:
def dct_matrix(n, dtype=torch.float32, device='cpu'):
    """
    构造 n x n 的 DCT-II 正交变换矩阵。
    
    参数:
        n: int, DCT 矩阵的大小
        dtype: PyTorch 数据类型（默认 float32）
        device: PyTorch 设备（默认 cpu）

    返回:
        DCT-II 类型的 n × n 正交变换矩阵
    """
    D = torch.zeros((n, n), dtype=dtype, device=device)
    for k in range(n):
        for i in range(n):
            alpha = math.sqrt(1.0 / n) if k == 0 else math.sqrt(2.0 / n)
            D[k, i] = alpha * math.cos(math.pi * (2 * i + 1) * k / (2 * n))
    return D

In [28]:
# DCT-II 变换
a = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
print("输入信号：")
print(a)
print("DCT-II 变换结果：")
D = dct_matrix(4, dtype=torch.float32)
print(D @ a)

输入信号：
tensor([1., 2., 3., 4.])
DCT-II 变换结果：
tensor([ 5.0000, -2.2304,  0.0000, -0.1585])


In [29]:
def dst_matrix(n, type=1, dtype=torch.float32, device='cpu'):
    """
    构造离散正弦变换（DST）矩阵，支持类型 I 和 II。
    
    参数：
        n (int): 变换矩阵的大小
        type (int): DST 类型（1 或 2）
        dtype: 数据类型（默认 float32）
        device: PyTorch 设备（默认 'cpu'）

    返回：
        n × n 的 DST 矩阵（float tensor）
    """
    if type not in [1, 2]:
        raise ValueError("Only DST type 1 and 2 are supported.")

    S = torch.zeros((n, n), dtype=dtype, device=device)
    if type == 1:
        for k in range(n):
            for i in range(n):
                S[k, i] = math.sqrt(2.0 / (n + 1)) * math.sin(math.pi * (k + 1) * (i + 1) / (n + 1))
    elif type == 2:
        for k in range(n):
            for i in range(n):
                alpha = math.sqrt(2.0 / n)
                S[k, i] = alpha * math.sin(math.pi * (2 * i + 1) * (k + 1) / (2 * n))
    return S

In [30]:
# DST   变换
a = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
print("输入信号：")
print(a)
print("DST-II 变换结果：")
S = dst_matrix(4, type=2, dtype=torch.float32)
print(S @ a)

输入信号：
tensor([1., 2., 3., 4.])
DST-II 变换结果：
tensor([ 4.6194, -2.0000,  1.9134, -1.4142])
